# Turn GitHub Schedule Project into Website Schedule.yaml
### Starter notebook (still room for more automation, generalization, and improvement)

In [ ]:
# vars to be filled out to run the script
org = 'snowex-hackweek' #GitHub organization where the project lives

GHuser = ''
GHtoken = ''

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import yaml

In [ ]:
def get_fr_results(res, array, fields):
    """
    Parse a requests.get response (res) to create a list (array) with info from the given fields (fields, list)
    """
    for res in res.json():
        array.append([res[field] for field in fields])

In [ ]:
# list all organization projects to get project ID
url = f'https://api.github.com/orgs/{org}/projects'
orgproj = requests.get(url)
projlist = []
get_fr_results(orgproj, projlist, ['name','id'])
projlist

In [ ]:
# idenfity which of the projects contains the current schedule
proj_id = projlist[0][1]

# Create an API request to get the project [columns]
url = f'https://api.github.com/projects/{proj_id}/columns'
headers = {"Accept": "application/vnd.github.v3+json"}
proj_cols = requests.get(url, headers=headers, auth=HTTPBasicAuth(GHuser, GHtoken))
print("Status code: ", proj_cols.status_code)

In [ ]:
# list out all the column names and GitHub IDs for the project
proj_cards = []
get_fr_results(proj_cols, proj_cards, ['name','id'])
proj_cards

In [ ]:
# Create an API request to get each column's cards and put into a dict
schedule_dict = {}

for card in proj_cards:
    card_id = card[1]
    url = f'https://api.github.com/projects/columns/{card_id}/cards'
    headers = {"Accept": "application/vnd.github.v3+json"}
    proj_col = requests.get(url, headers=headers, auth=HTTPBasicAuth(GHuser, GHtoken))
    print("Status code: ", proj_col.status_code)
    
    hold_note=[]
    get_fr_results(proj_col, hold_note, ['note'])
    schedule_dict[card[0]] = hold_note

In [ ]:
schedule_dict

In [ ]:
# turn the json schedule dict into a dict of dicts with formatted strings for turning into yaml
fmted_sched_dict = {}
# iterate through each day in the dict to parse the strings
for day in schedule_dict:
    fmted_sched_dict[day] = []
    # iterate through each item in that day's list of schedule items
    for sched_item_str in schedule_dict[day]:
        # parse out the string into a dict of details for each event (ev_dets)
        ev_items = sched_item_str[0].split('\r\n')
        ev_dets = {}
        for item in ev_items:
            item_parts = item.split('**', 2)
            if len(item_parts) == 1:
                ev_dets["description"] = item_parts[0]
            elif item_parts[2] == '':
                ev_dets["title"] = item_parts[1]
            elif "Leads" in item_parts[1]:
                ev_dets[item_parts[1].split(':')[0]] = list(name.strip() for name in item_parts[2].split(','))
            elif "Duration" in item_parts[1]:
                ev_dets['time'] = item_parts[2]
            else:
                ev_dets[item_parts[1].split(':')[0]] = item_parts[2]
        fmted_sched_dict[day].append(ev_dets)

In [ ]:
fmted_sched_dict

In [ ]:
class DaySched(yaml.YAMLObject):
    def __init__(self, title, date, contents):
        self.title = title
        self.date = date
        self.contents = contents
        
    def __repr__(self):
         return "%s(title=%r, date=%r, contents=%r)" % (
            self.__class__.__name__, self.title, self.date, self.contents)
        
class DayItem(yaml.YAMLObject):
    def __init__(self, time, location, title, leads, description):
        self.time = time
        self.location = location
        self.title = title
        self.leads = leads
        self.description=description


# sched_yaml = yaml.dump(DaySched('Day 1', 'Mon 11 Jul', DayItem("9:30-10:30", 
#                                                                "Conference Room A", 
#                                                                "SnowEx is Awesome", 
#                                                                ["Anthony Arendt","Jessica Scheick"], 
#                                                                "we are making the schedule")
#                                ))

# sched_yaml = yaml.dump(DaySched('Day 1', 'Mon 11 Jul', fmted_sched_dict['Day 1']), sort_keys=False)



In [ ]:
manual_sched_dict = {"timezone": "UTC -7 (Pacific Daylight Time)",
                    "tabs": {DaySched('Day 1', 'Mon 11 Jul', fmted_sched_dict['Day 1']),
                            DaySched('Day 2', 'Tues 12 Jul', fmted_sched_dict['Day 2']),
                            DaySched('Day 3', 'Wed 13 Jul', fmted_sched_dict['Day 3']),
                            DaySched('Day 4', 'Thurs 14 Jul', fmted_sched_dict['Day 4']),
                            DaySched('Day 5', 'Fri 15 Jul', fmted_sched_dict['Day 5'])}
                    }

In [ ]:
sched_yaml = yaml.dump(manual_sched_dict, sort_keys=False)
print(sched_yaml)

### Ideas and Goals for future automation:
1. convert durations into actual times
2. use the times and some simple if statements (if "start_time" near noon, add lunch; if "running_time" of events so far >90 minutes, add a break) to add other schedule elements
3. Create a GitHub project template (perhaps with the website?) to make filling in cards/populating the initial schedule easier.
4. Generalize/functionize (or turn into a script) this notebook?
5. Save the schedule as the yaml
6. GitHub [manual] action that runs this and submits a PR?